In [11]:
import cv2
import numpy as np
import datetime
import os
from tensorflow.keras.models import load_model
import winsound
from gtts import gTTS
import playsound

# -----------------------------
# Settings
# -----------------------------
model = load_model("face_mask_detector.h5")
class_names = ["with_mask", "without_mask", "improper_mask"]

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Create folders if not exist
os.makedirs("snapshots", exist_ok=True)
os.makedirs("logs", exist_ok=True)

log_file = "logs/detection_log.txt"

# Voice alert function
def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = "alert.mp3"
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224)) / 255.0
        face = np.expand_dims(face, axis=0)

        pred = model.predict(face)
        class_id = np.argmax(pred)
        label = class_names[class_id]
        confidence = round(np.max(pred) * 100, 2)

        label_text = f"{label} ({confidence}%)"
        color = (0,255,0) if label=="with_mask" else (0,0,255)

        cv2.putText(frame, label_text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)

        # -------------------------
        # Alerts & Evidence System
        # -------------------------
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        if label != "with_mask":
            # Beep alert
            winsound.Beep(1200, 300)

            # Save snapshot
            img_name = f"snapshots/{label}_{timestamp}.jpg"
            cv2.imwrite(img_name, frame)

            # Log event
            with open(log_file, "a") as f:
                f.write(f"{timestamp} - {label} - {confidence}%\n")

            # Voice alert once
            speak("Please wear a mask")

    cv2.imshow("Mask Detection System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("✅ Program Finished & Webcam Closed")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
✅ Program Finished & Webcam Closed


In [2]:
python webcam_test.py


SyntaxError: invalid syntax (3173293136.py, line 1)